In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
from collections import abc

import numpy as np
import scipy
from matplotlib import pyplot as plt

from harmonic_balance import aft, freq, utils
from harmonic_balance import arclength_continuation as alc

ndarray = np.ndarray

In [ ]:
NH = 4
n = 2
N = 18
m1, m2 = 1, 2
c1, c2, c3 = 1, 0.5, 2
k1, k2, k3 = 1, 0.5, 2

M = np.diag([m1, m2])
C = np.array([[c1 + c2, -c2], [-c2, c2 + c3]])
K = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])

omega = 2
t0, tf = tlim = 0, 2 * np.pi / omega
tn = NH * 2
tls = np.linspace(*tlim, tn)
A = freq.get_A(omega, NH, M, C, K)

ks = (1, 1)
dofs = (0, 1)
is_cosines = (1, 0)
coefficients = (2, 1)
b_ext = freq.get_b_ext(NH, n, ks, dofs, is_cosines, coefficients)


def get_slice(dof: int, N: int):
    return np.s_[dof * N : (dof + 1) * N]


def get_nonlinear(
    factor: float,
) -> tuple[
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
]:
    dof = 0
    s_ = lambda N: get_slice(dof, N)

    def f_nl(x: ndarray, xdot: ndarray, N: int = 1) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * x[s_(N)] ** 3
        return res

    def df_nl_dx(x: ndarray, xdot: ndarray, N: int = 1) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * 3 * x[s_(N)] ** 2
        return np.diag(res.ravel())

    def df_nl_d_xdot(x: ndarray, xdot: ndarray, N: int = 1) -> ndarray:
        shape = x.shape[0]
        return np.zeros((shape, shape), dtype=float)

    return f_nl, df_nl_dx, df_nl_d_xdot


f_nl, df_nl_dx, df_nl_d_xdot = get_nonlinear(0.1)

x = np.array([2, 3])
zeros = np.zeros_like(x)
f_nl(x, zeros), df_nl_dx(x, zeros), df_nl_d_xdot(x, zeros)

(array([0.8, 0. ]),
 array([[1.2, 0. ],
        [0. , 0. ]]),
 array([[0., 0.],
        [0., 0.]]))

In [ ]:
z1, z0 = b_ext.toarray() + 0.01, b_ext.toarray()
omega1, omega0 = 2.01, 2.0
s = 0.1

(
    alc.get_P(z1, z0, omega1, omega0, s),
    alc.get_dP_dz(z1, z0),
    alc.get_dP_d_omega(omega1, omega0),
)

(np.float64(-0.008900000000000005),
 array([0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j]),
 0.019999999999999574)

In [ ]:
alc.get_db_dz(omega, z1, df_nl_dx, df_nl_d_xdot, NH, n, N)

array([[0.07280167+0.j, 0.        +0.j, 0.07280167+0.j, 0.        +0.j,
        0.07280167+0.j, 0.        +0.j, 0.07280167+0.j, 0.        +0.j,
        0.07280167+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j],
       [0.07280167+0.j, 0.        +0.j, 0.07280167+0.j, 0.        +0.j,
        0.07280167+0.j, 0.        +0.j, 0.07280167+0.j, 0.        +0.j,
        0.07280167+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j],
       [0.07280167+0.j, 0.        +0.j, 0.07280167+0.j, 0.        +0.j,
        0.07280167+0.j, 0.        +0.j, 0.07280167+0.j, 0.        +0.j,
        0.07280167+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
   

In [ ]:
alc.get_R(z1, omega1, f_nl, b_ext, NH, n, N, M, C, K)

array([ 0.06071849+0.j    ,  0.02      +0.j    , -4.02228251+3.2851j,
        1.951698  +2.3253j, -0.02008351+0.0402j, -0.141604  +0.0804j,
       -0.06048451+0.0603j, -0.222406  +0.1206j, -0.10088551+0.0804j,
       -0.303208  +0.1608j])